In [1]:
from langchain.messages import (
    HumanMessage,
)
import asyncio
from src.core.utils.logger import logger


Loaded environment from /home/nhatminh/Desktop/IsNex/envs/.env.dev
2026-01-07T17:10:58.496420Z [info     ] Logger initialized             [src.core.utils.logger] environment=development filename=logger.py func_name=<module> lineno=280 module=logger pathname=/home/nhatminh/Desktop/IsNex/apps/documents/src/core/utils/logger.py


In [2]:
from pprint import pprint
from langchain.agents.middleware import before_model, after_model, AgentState
from langchain.messages import AIMessage
from langgraph.runtime import Runtime
from typing import Any


@before_model(can_jump_to=["end"])
def check_message_limit(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    if len(state["messages"]) >= 50:
        return {
            "messages": [AIMessage("Conversation limit reached.")],
            "jump_to": "end"
        }
    return None

@after_model
def log_response(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    print("======================== STATE ========================")
    pprint(state)
    print("======================== STATE ========================")
    print("======================== RUNTIME ========================")
    pprint(runtime)
    print("======================== RUNTIME ========================")
    return None

In [4]:
from langchain.agents import create_agent
from src.agent.model_catalog import AgentContext, ModelCatalog, SupportedModels
from src.agent.model_catalog import model_selector
from src.agent.tools import tools


model = ModelCatalog.get_model(model_name=SupportedModels.DEEPSEEK_V3_1_NEX_N1_FREE)
agent = create_agent(
    model,
    tools=tools,
    system_prompt="You are a helpful assistant. Be concise and accurate. Only use tools when you realy need it.",
    middleware=[model_selector],  # type: ignore
    context_schema=AgentContext,
)


In [5]:
message =HumanMessage(content="hello, which large language model are you, who created you?")

In [6]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
# import asyncio if needed elsewhere

from src.agent.model_catalog import SupportedModels

async def generate(
    message: HumanMessage,
    model_name: str
):
    try:
        print("================ 2 ================")
        
        # mlflow_langchain.autolog()
        config: RunnableConfig = {
            "configurable": {"model_name": model_name}
        }
        
        for chunk in agent.stream(
            {"messages": [message]},
            stream_mode="messages",
            context=AgentContext(model_name=model_name),
        ):
            
            yield chunk

    except asyncio.CancelledError:
        logger.warning("Stream cancelled by client")
    except Exception as e:
        logger.error(f"Error during streaming: {e}", exc_info=True)

In [8]:


async for chunk in generate(message, model_name=SupportedModels.GROK_4_1_FAST):
    print("================ 1 ================")
    print(chunk)

================ 2 ================
================ 3 ================
======================== REQUEST ========================
ModelRequest(model=ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x7b8a94af1010>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7b8a94af1a90>, root_client=<openai.OpenAI object at 0x7b8a94fd6120>, root_async_client=<openai.AsyncOpenAI object at 0x7b8a94af17f0>, model_name='nex-agi/deepseek-v3.1-nex-n1:free', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1'),
             messages=[HumanMessage(content='hello, which large language model are you, who created you?', additional_kwargs={}, response_metadata={}, id='4b491b8f-9013-4b51-9a62-9e42592207e4')],
             system_message=SystemMessage(content='You are a helpful assistant. Be concise and accurate. Only use tools when you realy need it.', additional_kwarg